# EuRoc Evaluation wrt GT

In [2]:
%matplotlib widget

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d 

import pandas as pd
import os
import pprint

from evo.core.trajectory import PoseTrajectory3D
from evo.tools import plot
from evo.core import metrics
from evo.core import sync

from pose_trajectory_evaluation import PoseTrajectoryEvaluation

%reload_ext autoreload
%autoreload 2

In [26]:
dataset_path = '/media/berlukas/Data/data/datasets/EuRoc/'
export_paths = ['MH_01', 'MH_02', 'MH_03']
exported_missions = ['c3d7929a77bba0160c00000000000000', '8fc4b6d277bba0160c00000000000000', 'c5043f496abba0160c00000000000000']
eval = PoseTrajectoryEvaluation()

est_trajectories = []
gt_trajectories = []

assert len(export_paths) == len(exported_missions)
for export_path, mission_id in zip(export_paths, exported_missions):
    est_traj_filename = dataset_path + 'MH_est.csv'
    gt_traj_filename = dataset_path + export_path + '_gt.csv'

    est_df = eval.get_mission_from_csv(est_traj_filename, mission_id)
    est_trajectories.append(eval.convert_df_to_traj(est_df))
    
    gt_df = eval.parse_trajectory_txt(gt_traj_filename, delimiter = ',')
    gt_trajectories.append(eval.convert_df_to_traj(gt_df))
    
assert len(gt_trajectories) == len(est_trajectories)
print(f'Loaded {len(gt_trajectories)} missions.')

Loaded 3 missions.


------------------------------------------------------

In [4]:
max_diff = 10

delta = 10
delta_unit = metrics.Unit.frames
all_pairs = False

In [5]:
def plot_evaluations_APE(data, ape_metric, ape_stats):
    # Trajectory plot
    fig = plt.figure(figsize=(8, 6), dpi=160)
    traj_by_label = {
        "estimate": data[1],
        "reference": data[0]
    }
    plot.trajectories(fig, traj_by_label, plot.PlotMode.xy)
    plt.show()

    # Trajectory plot
    seconds_from_start = [t - data[1].timestamps[0] for t in data[1].timestamps]
    fig = plt.figure(figsize=(8, 6), dpi=160)
    plot.error_array(fig.gca(), ape_metric.error, x_array=seconds_from_start,
                     statistics={s:v for s,v in ape_stats.items() if s != "sse"},
                     name="APE", title="APE w.r.t. " + ape_metric.pose_relation.value, xlabel="$t$ (s)")
    plt.show()

    plot_mode = plot.PlotMode.xy
    fig = plt.figure(figsize=(8, 6), dpi=160)
    ax = plot.prepare_axis(fig, plot_mode)
    plot.traj(ax, plot_mode, gt_traj, '--', "gray", "reference")
    plot.traj_colormap(ax, data[1], ape_metric.error, 
                       plot_mode, min_map=ape_stats["min"], max_map=ape_stats["max"])
    ax.legend()
    plt.show()

def perform_evaluation_using_data(data, delta, delta_unit, all_pairs):
    print('--- Translational Part -----------------------------')
    pose_relation = metrics.PoseRelation.translation_part
    print('APE:')
    ape_metric_trans = metrics.APE(pose_relation)
    ape_metric_trans.process_data(data)
    ape_stat_trans = ape_metric_trans.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_trans = ape_metric_trans.get_all_statistics()
    pprint.pprint(ape_stats_trans)

#     print('RPE:')
#     rpe_metric_trans = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
#     rpe_metric_trans.process_data(data)
#     rpe_stat_trans = rpe_metric_trans.get_statistic(metrics.StatisticsType.rmse)
#     rpe_stats_trans = rpe_metric_trans.get_all_statistics()
#     pprint.pprint(rpe_stats_trans)
#     print('\n\n')

    print('--- Rotational Part -----------------------------')
    pose_relation = metrics.PoseRelation.rotation_angle_deg
    print('APE:')
    ape_metric_rot = metrics.APE(pose_relation)
    ape_metric_rot.process_data(data)
    ape_stat_rot = ape_metric_rot.get_statistic(metrics.StatisticsType.rmse)
    ape_stats_rot = ape_metric_rot.get_all_statistics()
    pprint.pprint(ape_stats_rot)

#     print('RPE:')
#     rpe_metric_rot = metrics.RPE(pose_relation, delta, delta_unit, all_pairs)
#     rpe_metric_rot.process_data(data)
#     rpe_stat_rot = rpe_metric_rot.get_statistic(metrics.StatisticsType.rmse)
#     rpe_stats_rot = rpe_metric_rot.get_all_statistics()
#     pprint.pprint(rpe_stats_rot)

    plot_evaluations_APE(data, ape_metric_trans, ape_stats_trans)

In [29]:
def ts_ns_to_seconds(ts_ns):
    k_ns_per_s = 1e9;
    return ts_ns / k_ns_per_s;

def convert_est_to_traj(trajectory):
    ts = ts_ns_to_seconds(trajectory[:,0])
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

def convert_gt_to_traj(trajectory):
    ts = trajectory[:,0]
    xyz = trajectory[:,1:4]
    wxyz = trajectory[:,4:8]
    return PoseTrajectory3D(positions_xyz = xyz, orientations_quat_wxyz = wxyz, timestamps = ts)

idx = 0

print(f'=== Maplab ===========================================================')
est_traj = convert_est_to_traj(est_trajectories[idx])
gt_traj = convert_est_to_traj(gt_trajectories[idx])
gt_traj, est_traj = sync.associate_trajectories(gt_traj, est_traj, max_diff)
est_traj.align(gt_traj, correct_scale=False, correct_only_scale=False)

data = (gt_traj, est_traj)
perform_evaluation_using_data(data, delta, delta_unit, all_pairs)

=== Maplab ===========================================================
--- Translational Part -----------------------------
APE:
{'max': 0.13168626378237247,
 'mean': 0.01604921009186454,
 'median': 0.013001668970449333,
 'min': 0.0017859881543530582,
 'rmse': 0.020270098942195156,
 'sse': 0.0723143363582431,
 'std': 0.012381428292146852}
--- Rotational Part -----------------------------
APE:
{'max': 179.95834608730019,
 'mean': 170.7740034846596,
 'median': 172.60737491404802,
 'min': 146.9440690879839,
 'rmse': 170.88955303402082,
 'sse': 5139770.123165466,
 'std': 6.283237222076427}


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

/usr/local/lib/python3.6/dist-packages/matplotlib/pyplot.py:522: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…